In [2]:
import pandas as pd

df = pd.read_csv(r"C:\Users\egeen\Downloads\SensorNamerefineddataset.csv")

df.head()


,Sensorp,date,time,Parking_Status
0,Sensor1,2/18/2021,0:00,0
1,Sensor1,2/18/2021,7:30,1
2,Sensor1,2/18/2021,8:00,1
3,Sensor1,2/18/2021,8:30,1
4,Sensor1,2/18/2021,9:00,1


In [3]:
# Convert `date_time` column into a datetime column
df[' date'] = pd.to_datetime(df[' date'], format='%m/%d/%Y')

# See data types of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84791 entries, 0 to 84790
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Sensorp         84791 non-null  object        
 1    date           84791 non-null  datetime64[ns]
 2   time            84791 non-null  object        
 3   Parking_Status  84791 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.6+ MB


In [4]:
df['time'] = pd.to_datetime(df['time'], format='%H:%M')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84791 entries, 0 to 84790
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Sensorp         84791 non-null  object        
 1    date           84791 non-null  datetime64[ns]
 2   time            84791 non-null  datetime64[ns]
 3   Parking_Status  84791 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 2.6+ MB


In [5]:
# Create new features from `date_time` column
df['year'] = df[' date'].dt.year
df['day'] = df[' date'].dt.day
df['month'] = df[' date'].dt.month
df['day_of_week'] = df[' date'].dt.weekday
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
df['date'] = df[' date'].dt.date
df['time'] = df['time'].dt.strftime('%H:%M')
df['hour_min'] = round(df['hour'] + (df['minute'] / 60), 1)

# Convert `month` to categorical
df['month'].replace({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
                     7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}, inplace=True)
df['month'] = df['month'].astype('category') 
df['month'].cat.set_categories(new_categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True, inplace=True)

# Convert `day_of_week` to categorical
df['day_of_week'].replace({0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}, inplace=True)
df['day_of_week'] = df['day_of_week'].astype('category') 
df['day_of_week'].cat.set_categories(new_categories=['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'], 
                                     ordered=True, inplace=True)

# Convert `hour` to categorical
df['hour'] = df['hour'].astype('category') 
df['hour'].cat.set_categories(new_categories=list(range(24)), ordered=True, inplace=True)

# Show first 5 rows
df.head()

C:\Users\egeen\anaconda3\lib\site-packages\pandas\core\arrays\categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


,Sensorp,date,time,Parking_Status,year,day,month,day_of_week,hour,minute,date,hour_min
0,Sensor1,2021-02-18,00:00,0,2021,18,Feb,Thu,0,0,2021-02-18,0.0
1,Sensor1,2021-02-18,07:30,1,2021,18,Feb,Thu,7,30,2021-02-18,7.5
2,Sensor1,2021-02-18,08:00,1,2021,18,Feb,Thu,8,0,2021-02-18,8.0
3,Sensor1,2021-02-18,08:30,1,2021,18,Feb,Thu,8,30,2021-02-18,8.5
4,Sensor1,2021-02-18,09:00,1,2021,18,Feb,Thu,9,0,2021-02-18,9.0


In [10]:
from datetime import datetime, timedelta
import requests

def parse_date(date_string):
    """Parse a date string in YYYY-MM-DD format as a datetime object."""
    
    return datetime.strptime(date_string, '%Y-%m-%d').date()

def get_ph_and_eve(year='2021'):
    """Return lists of parsed dates for Singapore's public holidays and public holday eves."""
    
    if year == '2021':
        url = 'https://data.gov.sg/api/action/datastore_search?resource_id=550f6e9e-034e-45a7-a003-cf7f7e252c9a&'
    
    elif year == '2022':
        url = 'https://data.gov.sg/api/action/datastore_search?resource_id=04a78f5b-2d12-4695-a6cd-d2b072bc93fe&'
    
    data = requests.get(url).json()
    ph_parsed = [parse_date(ele['date']) for ele in data['result']['records']]
    eve_parsed = [parse_date(ele['date']) - timedelta(days=1) for ele in data['result']['records']]
    
    return ph_parsed, eve_parsed

ph_parsed, eve_parsed = get_ph_and_eve(year='2021')
print(f'2021 Public Holidays: {ph_parsed}')
print()
print(f'2021 Public Holiday Eves: {eve_parsed}')

2021 Public Holidays: [datetime.date(2021, 1, 1), datetime.date(2021, 2, 12), datetime.date(2021, 2, 13), datetime.date(2021, 4, 2), datetime.date(2021, 5, 1), datetime.date(2021, 5, 13), datetime.date(2021, 5, 26), datetime.date(2021, 7, 20), datetime.date(2021, 8, 9), datetime.date(2021, 11, 4), datetime.date(2021, 12, 25)]

2021 Public Holiday Eves: [datetime.date(2020, 12, 31), datetime.date(2021, 2, 11), datetime.date(2021, 2, 12), datetime.date(2021, 4, 1), datetime.date(2021, 4, 30), datetime.date(2021, 5, 12), datetime.date(2021, 5, 25), datetime.date(2021, 7, 19), datetime.date(2021, 8, 8), datetime.date(2021, 11, 3), datetime.date(2021, 12, 24)]
